# Answering Business Questions using SQL

The goal of this project is to explore the Chinook Database, a fake online music store database, and answer some questions using the data from it. 

Let's first connect and load in the database:

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

## Overview of the Data

We'll explore the name and type for each database in the database. 

In [2]:
%%sql
SELECT 
    name,
    type
  FROM sqlite_master
  WHERE type IN ('table', 'view');

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [3]:
%%sql
SELECT *
  FROM invoice
  LIMIT 5;

Done.


invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84
2,30,2017-01-03 00:00:00,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,9.9
3,40,2017-01-05 00:00:00,"8, Rue Hanovre",Paris,None,France,75002,1.98
4,18,2017-01-06 00:00:00,627 Broadway,New York,NY,USA,10012-2612,7.92
5,27,2017-01-07 00:00:00,1033 N Park Ave,Tucson,AZ,USA,85719,16.83


From further investigation of the tables we come to this schema of how they all connect together.

In [14]:
# import image module
from IPython.display import Image
  
# get the image
Image(url="https://s3.amazonaws.com/dq-content/191/chinook-schema.svg", width=700, height=500)



## Selecting Albums to Purchase

One of the questions for this data was to figure out which 3 albums from new artists with no current music on the platform should be added to the US store from a list of four albums:


|Artist |Genre|
|---|---|
|Regal|	Hip-Hop|
|Red Tone|	Punk|
|Meteor and the Girls|	Pop|
|Slim Jim Bites|Blues|


In order to figure out which albums to purchase and promote we'll check which genres are more popular and see if that can drive our decision.

We'll first make a subquery of which tracks are from the US and from there run a query that joins `track`, `genre`, `invoice_line_id` that gives us the genre, how many tracks have sold, and what percentage the genre makes of sales. 

In [4]:
%%sql
WITH 
     usa_tracks AS
    (
    SELECT invl.*
      FROM invoice_line invl
      INNER JOIN invoice inv ON inv.invoice_id = invl.invoice_id
      INNER JOIN customer c ON inv.customer_id = c.customer_id
      WHERE c.country = 'USA'
    )
    
SELECT 
    g.name genre,
    COUNT(ut.invoice_line_id) tracks_sold,
    CAST(COUNT(ut.invoice_id) AS FLOAT) / (
         SELECT COUNT(*) FROM usa_tracks
    ) percentage_sold
    FROM usa_tracks ut
    INNER JOIN track t ON t.track_id = ut.track_id
    INNER JOIN genre g ON g.genre_id = t.genre_id
    GROUP BY 1
    ORDER BY 3 DESC;

Done.


genre,tracks_sold,percentage_sold
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Latin,22,0.02093244529019981
Pop,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


## Which Artist to Sign

Based off the above sql query the most sucessful genres that correspond to the four avaliable albums would be as follows:

1. Red Tone, Punk
2. Slim Jim Bites, Blues
3. Meteor and the Girls, Pop

These would be the choices to pick based off of the data avaliable to us currently with the genres making up a combined 17.9%. However, we should be on the lookout for any Rock artists to sign since they make up a majority of track sales (53%).

## Analyzing Employee Sales Performance

Each customer from the Chinook store gets assigned to a sales support agent when making their first purchase. We want to now analyze if those employees have any significant differences in the amount of sales they bring into Chinook. 

We'll first make a subquery of customers total purchases and who their customer rep is by id number. Then we'll get the names of them from the `employee` table and group them by their name and see what their total sales were and also show their hire date. 

In [5]:
%%sql

 SELECT inv.customer_id, c.support_rep_id, SUM(inv.total) total
    FROM customer c
    INNER JOIN invoice inv ON inv.customer_id = c.customer_id
    INNER JOIN employee e ON e.employee_id = c.support_rep_id
    GROUP BY 1,2
    ORDER BY 1;

Done.


customer_id,support_rep_id,total
1,3,108.89999999999998
2,5,82.17
3,3,99.99
4,4,72.27000000000001
5,4,144.54000000000002
6,5,128.7
7,5,69.3
8,4,60.38999999999999
9,4,37.61999999999999
10,4,60.39


In [6]:
%%sql

WITH 
     customer_support_rep_sales AS
    (
    SELECT inv.customer_id, c.support_rep_id, SUM(inv.total) total
    FROM customer c
    INNER JOIN invoice inv ON inv.customer_id = c.customer_id
    INNER JOIN employee e ON e.employee_id = c.support_rep_id
    GROUP BY 1,2
    ORDER BY 1
    )
    
SELECT
      e.first_name || " " || e.last_name employee,
      SUM(csrs.total) total_purchases,
      e.hire_date
    FROM customer_support_rep_sales csrs
    INNER JOIN employee e ON e.employee_id = csrs.support_rep_id
    GROUP BY 1;

Done.


employee,total_purchases,hire_date
Jane Peacock,1731.5099999999998,2017-04-01 00:00:00
Margaret Park,1584.0000000000002,2017-05-03 00:00:00
Steve Johnson,1393.92,2017-10-17 00:00:00


There are three employees who respresent all of the customers of Chinook and have their customer's assigned to them have made total purcahse amounts of:

1. Jane Peacock, 1731.51
2. Margaret Park, 1584
3. Steve Johnson, 1393.92

Their hiring dates also correspond with the ordering of total purchases as the longest hire, Jane Peacock, has the most sales and Steve Johnson, the newest hire, has the least sales.

## Analyzing Sales by Country

Now we'll analyze the sales of music by country (from the `customers` table and not the `invoice` billing address) and see what information it reveals about our customer base.

We'll first need to make a query which returns a table of each purchase and which country it came from along with the track and invoice information. For any country where there is only one purchase we will categorize it as `Other` in the table.

In [16]:
%%sql
SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       il.*
     FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
     LIMIT 20

Done.


country,customer_id,invoice_line_id,invoice_id,track_id,unit_price,quantity
USA,18,1,1,1158,0.99,1
USA,18,2,1,1159,0.99,1
USA,18,3,1,1160,0.99,1
USA,18,4,1,1161,0.99,1
USA,18,5,1,1162,0.99,1
USA,18,6,1,1163,0.99,1
USA,18,7,1,1164,0.99,1
USA,18,8,1,1165,0.99,1
USA,18,9,1,1166,0.99,1
USA,18,10,1,1167,0.99,1


Now we'll use that query (removing `LIMIT 20`) and group by country with totals of sales, total customers, total sales, average order number, and average sales amount per customer.

In [8]:
%%sql

WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       il.*
     FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
    )
    
SELECT
    country,
    total_customers,
    total_sales,
    avg_order,
    avg_sale_per_customer
FROM
  (
  SELECT 
     country,
      COUNT(DISTINCT customer_id) total_customers,
      SUM(unit_price) total_sales,
      (SUM(unit_price) / COUNT(DISTINCT customer_id)) avg_sale_per_customer,
      (SUM(unit_price) / COUNT(DISTINCT invoice_id)) avg_order,
      CASE
        WHEN country = 'Other' THEN 1
        ELSE 0
        END AS sort
    FROM country_or_other
    GROUP BY 1
    ORDER BY sort ASC, total_sales DESC
  );

Done.


country,total_customers,total_sales,avg_order,avg_sale_per_customer
USA,13,1040.490000000008,7.942671755725252,80.03769230769292
Canada,8,535.5900000000034,7.047236842105309,66.94875000000043
Brazil,5,427.68000000000245,7.011147540983647,85.53600000000048
France,5,389.0700000000021,7.781400000000042,77.81400000000042
Germany,4,334.6200000000016,8.161463414634186,83.6550000000004
Czech Republic,2,273.24000000000103,9.108000000000034,136.62000000000052
United Kingdom,3,245.52000000000078,8.768571428571457,81.84000000000026
Portugal,2,185.13000000000022,6.3837931034482835,92.56500000000011
India,2,183.1500000000002,8.72142857142858,91.5750000000001
Other,15,1094.9400000000085,7.448571428571486,72.99600000000056


If we were to choose a couple countries to spend advertising money on we could choose either a couple of the countries with lower amounts of customers (United Kingdom, India, etc.) or we could advertise to our biggest markets (US and Canada) to increase sales.

This is all just speculative though as the number of customers for this service is pretty low and thus we analyzing using a small sample size. Because of this, we should be cautious spending too much money on new marketing campaigns, as the sample size is not large enough to give us high confidence. 

## Albums vs Individual Tracks

The way Chinook works is that customers can either purchase a whole album or buy tracks from said album piece meal (unless manually added on after purchasing the album). Chinook is considering to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

In order to see if this would save costs we'll look at the percentage of purchases of individual tracks versus purchases of whole albums. Looking at the invoices of each purchase and comparing the track id numbers to the original album they are purchased from should give us an idea of what happens more often.

We first need to build a query to check on an invoice and give the id of the invoice along with the first tracks id number.

In [17]:
%%sql
SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
     LIMIT 20;

Done.


invoice_id,first_track_id
1,1158
2,201
3,2516
4,748
5,1986
6,30
7,42
8,81
9,196
10,2663


Now we can use that query and compare if an invoice was a whole album or wasn't using `EXCEPT`.

In [10]:
%%sql

WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )


SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(COUNT(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;
    

Done.


album_purchase,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674


With 18.6% of sales being whole albums we would recommend not purchasing tracks piece meal from record companies as it could lead to 1/5 of revenue lost.